<a href="https://colab.research.google.com/github/maxfloresv/FM849/blob/feat%2Fadd-20252-material/20252/tutorials/02_exploratory_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FM849 Proyecto de Ciencia de Datos: Inteligencia Artificial (IA) y sus aplicaciones

- **Profesor de Cátedra:** Máximo Flores Valenzuela ($\texttt{mflores@dcc.uchile.cl}$).
- **Profesor Auxiliar:** Héctor Jiménez Orellana

## 📚 Cátedra III: Análisis exploratorio de datos

Bienvenidos/as al segundo tutorial del curso. En este, veremos cómo analizar los datos desde una perspectiva exploratoria, lo que permite entender con qué estamos trabajando y responder las preguntas que vimos en la tercera cátedra ([03_data-analysis.pdf](https://www.u-cursos.cl/escverano/2025/1/FM849/1/material_docente/detalle?id=9495749)).

### Carga y lectura de datos

Tal como en el final del [tutorial I](https://colab.research.google.com/drive/1HcIlErvfLMXOXKgRwqAMoa0oWMpWe1nq?usp=sharing#scrollTo=lLpCl4HUMV4H), lo primero que haremos será cargar y leer los datos.


In [ ]:
%%capture

!wget https://users.dcc.uchile.cl/~mflores/FM849/tutorial1_ds.csv

Noten que estamos usando un _encoding_ de tipo $\texttt{cp1252}$. Esta es una alternativa a $\texttt{utf-8}$, la codificación que viene por defecto. Hay algunos _datasets_ que no son compatibles con $\texttt{utf-8}$ y hay que ir probando distintas codificaciones según el formato que tengan los datos.

In [ ]:
import pandas as pd
# df viene de "DataFrame"
df = pd.read_csv("tutorial1_ds.csv", encoding='cp1252')

df.head()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


### Análisis estadístico

Para realizar un análisis estadístico, la primera operación que podemos realizar es ver cuántos datos se cargaron. Para ello, usaremos la siguiente propiedad `shape`, que nos entrega dos valores: la cantidad de filas, y la cantidad de columnas (en ese orden).

In [ ]:
df.shape

(1218, 11)

En este caso, tenemos pocos datos ($1.218$). Esto es una limitante si quieren resolver un problema que deba involucrar muchos datos, porque el tener una muestra muy específica introduce problemas de sesgo. Piensen en el siguiente escenario:

> **Una persona quiere caracterizar a los usuarios de bicicleta que se mueven por Santiago. Su muestra es de $500$ personas, todas pasan por el cerro San Cristóbal. ¿Es suficiente la muestra de $500$ personas para concluir algo en el estudio?**

La respuesta es no, porque la muestra es muy específica e introduce lo que se llama «sesgo de selección». Otro ejemplo claro es el de las encuestas para un trabajo: ¿qué pasa si publico en un foro _online_ un enlace a una encuesta creada con Google Forms? ¡Aquí se introduce el «sesgo de autoselección» (nótese que se agrega el prefijo «auto» al inicio)!, pues generalmente las personas que están dispuestas a responder la encuesta por algún motivo (p. ej., incentivo económico) la responderán, y el resto no. Ese resto puede contar una historia que cambie lo que cuentan los datos actuales.

Cuando nos enfrentamos a esta limitación, tenemos opciones:

- Cambiar el _dataset_. No tengan miedo de hacerlo si están realizando algún proyecto donde no están obligados a usar un _dataset_ en específico.
- Buscar _datasets_ complementarios y realizar una unión. Muchos datos ya están disponibles en internet. Pueden complementarlo con lo que tienen.
- Aplicar una técnica llamada _Data Augmentation_ (pueden revisar más información si les interesa en el siguiente enlace: [(_click_ acá)](https://aws.amazon.com/es/what-is/data-augmentation/)). Usualmente se hace cuando las clases de nuestras observaciones están desbalanceadas.

Si usamos `describe()`, podemos obtener información sobre el _dataset_ en vista general. Como todas las variables son de tipo $\texttt{object}$ por defecto, deberían ver algo como lo que sigue, y en cuestión:

- `count` muestra un conteo de los valores no nulos de cada columna. Un valor es nulo si viene vacío.
- `unique` muestra un conteo de los valores únicos no nulos de cada columna. Sirve para ver cuánta variabilidad tenemos en los datos. Si tenemos pocos valores únicos, podemos evaluar eliminar la columna, dependiendo de la importancia que tenga en nuestro proyecto.
- `top` muestra la moda de la columna, es decir, el valor que más se repite.
- `freq` muestra la frecuencia de la moda.

In [ ]:
df.describe()

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
count,1218,1218,1218,1215,1218,1218,1212,1218,1218,1218,1217
unique,37,1201,356,311,456,114,180,535,23,19,263
top,Nissan,Mistral,I4,1984 cc,355 hp,250 km/h,6.5 sec,"$35,000",Petrol,5,400 Nm
freq,149,2,64,31,23,145,45,36,871,692,72


### Procesamiento previo al EDA

Como preferencia personal, me gusta renombrar las columnas a nombres más compactos, que obviamente, sigan siendo descriptivos. Haremos esto con la siguiente estandarización: los nombres pasarán a estar en minúsculas. Esto es con el fin de no repetir textos largos al seleccionar columnas. La instrucción `inplace=True` hace que la modificación se haga sobre `df`, y no devuelva una copia. Noten que la instrucción para hacer esto es `rename`, y recibe un argumento `columns` con un diccionario que realiza las traducciones.

In [ ]:
df.rename(columns={
    "Company Names": "company_name",
    "Cars Names": "car_name",
    "Engines": "engines",
    "CC/Battery Capacity": "battery_capacity",
    "HorsePower": "hp",
    "Total Speed": "total_speed",
    "Performance(0 - 100 )KM/H": "performance",
    "Cars Prices": "prices",
    "Fuel Types": "fuel_types",
    "Seats": "seats",
    "Torque": "torque"
  }, inplace=True)

df.head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


Ahora, notemos lo siguiente: hay columnas que sí o sí son cadenas de texto, como `company_name`, `car_name`, `engines` y `fuel_types`. Pongámosles explícitamente ese tipo de dato, que por defecto viene como `object` como mencionamos anteriormente:

In [ ]:
str_cols = ["company_name", "car_name", "engines", "fuel_types"]
for col in str_cols:
    df[col] = df[col].astype("string") # string := cadena de texto

df.dtypes # dtypes := data types

,0
company_name,string[python]
car_name,string[python]
engines,string[python]
battery_capacity,object
hp,object
total_speed,object
performance,object
prices,object
fuel_types,string[python]
seats,object


#### Capacidad de la batería

Nos falta dilucidar qué tipo de dato le ponemos a las columnas faltantes. En `battery_capacity`, todos los datos parecieran estar sufijados por $\texttt{cc}$. Veamos si esto realmente es así aplicando una expresión regular (les puede ser útil revisar esta página subida a Material Docente: [RegExr](https://www.u-cursos.cl/escverano/2025/1/FM849/1/enlaces/detalle?id=1037341), y apoyarse de LLMs como GPT). También, hay que deshacerse de la coma, pues hay datos que tienen este separador, y otros que no. Otra alternativa es ponerle comas a todos.

In [ ]:
# Esta RegEx fue generada con ChatGPT 5.
pat = r'^\s*(?:\d+|\d{1,3}(?:,\d{3})+)(?:\.\d+)?\s*cc\s*$'
match_condition = df["battery_capacity"].str.fullmatch(
    pat, case=False, na=False
  )

# ~ -> not := no cumple la condición.
df[~match_condition].head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
97,MERCEDES,BENZ EQS 53,ELECTRIC MOTOR,NaN,751 hp,250 km/h,3.4 sec,"$102,000",Electric,5,950 Nm
139,AUDI,E-TRON,ELECTRIC,95 kwh,402 hp,200 km/h,5.5 sec,"$75,000",Electric,5,664 Nm
140,AUDI,E-TRON SPORTBACK,ELECTRIC,95 kwh,402 hp,200 km/h,5.5 sec,"$78,000",Electric,5,664 Nm
142,BMW,I8 ROADSTER,ELECTRIC MOTOR,11.6 kwh,369 hp,250 km/h,4.4 sec,"$165,000",Hybrid,2,570 Nm
183,KIA,SPORTAGE PHEV,I4 + ELECTRIC,"1,600 cc / 13.8 kwh",261 hp,180 km/h,8.0 sec,"$41,000",Plug-in Hybrid,5,350 Nm


La tabla de arriba tiene $5$ entradas que no cumplen el patrón inducido por la expresión regular de `pat`. Es interesante ver cuántos son en total:

In [ ]:
df[~match_condition].shape

(227, 11)

Ya encontramos problemas: tenemos valores nulos ($\texttt{NaN}$), y además, hay formatos distintos (p. ej., que usan $\texttt{kwh}$ en vez de $\texttt{cc}$). En total, son $227$ las observaciones que no tienen el formato terminado en $\texttt{cc}$ que discutimos arriba.

Primero, revisaremos cuántas de estas observaciones tienen un valor nulo, usando el método `isna()`:

In [ ]:
df[df['battery_capacity'].isna()].shape

(3, 11)

Son sólo 3, entonces ahora tenemos las siguientes opciones:

- Eliminar las observaciones asociadas a los valores nulos: sólo si de verdad no me aportan nada. A veces los valores nulos sí cuentan alguna historia (p. ej., preguntas no respondidas en una entrevista).
- Reemplazar dichas observaciones por algún valor, por ejemplo, la media $\overline{X}$, la desviación estándar $\sigma$, etc.: mucho cuidado, porque es una práctica muy común, pero los datos resultantes deben tener algún sentido.

En este caso, vamos a mostrar primero cuáles son esas columnas, y analizarlas. Dado que son sólo $3$, no necesitamos usar `head()` (aunque es buena práctica usarlo para no ensuciar los _outputs_, salvo que se quieran analizar más observaciones):

In [ ]:
df[df['battery_capacity'].isna()]

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
97,MERCEDES,BENZ EQS 53,ELECTRIC MOTOR,NaN,751 hp,250 km/h,3.4 sec,"$102,000",Electric,5,950 Nm
255,Nissan,Urvan,2.5L Turbo Diese,NaN,2488 cc,140 km/h,14.0 sec,"$28,000",Diesel,15,356 Nm
1147,Mazda,RX-Vision Concept,Rotary SkyActiv-R,NaN,400 hp (est.),300 km/h (est.),NaN,N/A (Concept Only),Petrol (Hybrid),2,NaN


La última observación ($1147$) tiene muchos valores nulos. No es una observación que nos aporte. Ni siquiera tiene precio y se menciona que sólo es de concepto. Con respecto a las otras observaciones ($97$ y $255$), podríamos, por ejemplo, buscar la información faltante en internet. Esto no siempre es una práctica viable, porque pueden ser incluso millones de datos los que vienen con valores vacíos.

Revisaremos primero la observación $97$, viendo si hay autos similares de Mercedes (como mínimo, buscando que la compañía sea la misma y el motor sea eléctrico):

In [ ]:
# El "y" lógico en pandas se lee "&".
df[
    (df["company_name"] == "MERCEDES")
    & (df["engines"] == "ELECTRIC MOTOR")
  ].head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
97,MERCEDES,BENZ EQS 53,ELECTRIC MOTOR,NaN,751 hp,250 km/h,3.4 sec,"$102,000",Electric,5,950 Nm


Como no hay ninguno más, podemos sospechar que nos faltan datos, o que Mercedes no produce muchos autos con motor eléctrico. Esto requiere un trabajo de investigación: primero viendo de cuándo es el _dataset_, y segundo, mirando cuántos autos con motor eléctrico había sacado Mercedes hasta esa fecha. Por ahora, eliminaremos la entrada para dar a conocer la instrucción:

In [ ]:
df.drop(df.index[97], inplace=True)
df.reset_index(drop=True, inplace=True)

En la celda de código de arriba, `df.index[97]` hace referencia a la observación con índice $97$ (es decir, el Mercedes eléctrico). La instrucción `df.drop(...)` borra esa fila, y posteriormente, `df.reset_index(drop=True)` reinicia los índices para que no haya un salto del $96$ al $98$. Si no se ocupa `drop=True`, se dejan los índices antiguos en una nueva columna.

Ahora, ya no debería existir dicha entrada:

In [ ]:
df[
    (df["company_name"] == "MERCEDES")
    & (df["engines"] == "ELECTRIC MOTOR")
  ].head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque


Para el caso de Nissan, lo que haremos será tomar el promedio de todas las capacidades de batería que estén asociadas a autos de dicha compañía. Aprovecharemos de usar la instrucción `notna()` para sólo sacar los valores no vacíos.

In [ ]:
nissan_mean = df[
    (df["company_name"] == "Nissan")
    & (df["battery_capacity"].notna())
  ]
nissan_mean.head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
285,Nissan,NV1500,4.0L V6,3954 cc,261 hp,160 km/h,9.5 sec,"$33,000",Petrol,12,385 Nm
380,Nissan,Versa,1.6L Inline-4,1598 cc,122 HP,180 km/h,10.4 sec,"$16,700",Petrol,5,154 Nm
381,Nissan,Micra,1.0L Turbocharged Inline-3,999 cc,100 HP,175 km/h,10.8 sec,"$15,500",Petrol,5,160 Nm
382,Nissan,Patrol Platinum,5.6L V8,5552 cc,400 HP,210 km/h,7.3 sec,"$77,000",Petrol,7,560 Nm
383,Nissan,Qashqai,1.3L Turbocharged Inline-4,1332 cc,156 HP,195 km/h,8.9 sec,"$26,000",Petrol,5,260 Nm


Para poder promediar, antes necesitamos saber: ¿habrá alguno que no cumple el patrón de dígitos más el texto $\texttt{cc}$?

In [ ]:
new_match_condition = nissan_mean["battery_capacity"].str.fullmatch(
    pat, case=False, na=False
  )
nissan_mean[~new_match_condition].shape

(6, 11)

Son sólo $6$ entradas, entonces podemos mostrarlas todas:

In [ ]:
nissan_mean[~new_match_condition]

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
402,Nissan,Ariya,Dual Electric Motors,87 kWh,389 HP,200 km/h,5.1 sec,"$58,950",Electric,5,600 Nm
417,Nissan,Ariya E-4ORCE,Dual Electric Motors,87 kWh,389 HP,200 km/h,5.1 sec,"$60,000",Electric,5,600 Nm
432,Nissan,Ariya Lounge Edition,Dual Electric Motors,87 kWh,389 HP,200 km/h,5.1 sec,"$63,000",Electric,5,600 Nm
448,Nissan,Ariya Premiere,Dual Electric Motors,87 kWh,389 HP,200 km/h,5.1 sec,"$67,000",Electric,5,600 Nm
469,Nissan,Ariya Empower+,Dual Electric Motors,87 kWh,389 HP,200 km/h,5.1 sec,"$68,000",Electric,5,600 Nm
492,Nissan,Ariya Dual Motor AWD,Dual Electric Motors,87 kWh,389 HP,200 km/h,5.1 sec,"$70,000",Electric,5,600 Nm


Aquí hay algo extraño y que suele pasar en los análisis exploratorios: ¿analizamos por qué los autos eléctricos pueden tener otra medida para `battery_capacity`? Si no están familiarizados con las unidades de medida, no hay una conversión directa entre $\texttt{kWh}$ y $\texttt{cc}$, entonces sí representan cosas distintas.

Si revisan la documentación que tiene el _dataset_ (👀: no siempre estará disponible), nos indica que la columna `battery_capacity` sí hace una distinción entre autos eléctricos y convencionales ([link al _dataset_](https://www.kaggle.com/datasets/abdulmalik1518/cars-datasets-2025)).

Dado esto, introduciremos otra forma de diagnosticar los valores de una tabla. Noten que las unidades de medida siempre son sufijos, es decir, están al final. Podemos ver cuáles son los sufijos únicos de todas las observaciones.

In [ ]:
df_sin_nulos = df[df['battery_capacity'].notna()]
separados = df_sin_nulos['battery_capacity'].str.split()
separados.apply(lambda x: x[-1]).unique()

array(['cc', 'kwh', 'Model', 'cc)', 'KWH', 'Range)', 'kWh', '(Estimate)',
       'Motor', 'Battery', '(V6)', '(I4)', '(Battery)', '(V8)',
       'Electric', '-', 'kWh)', '(Electric)', 'battery', 'batt'],
      dtype=object)

En este bloque de código partimos quitando los valores nulos, porque no podemos tratarlos como listas para acceder al último elemento, que es lo que queremos lograr para extraer el sufijo.

La variable `separados` guarda cada observación, exclusivamente de la columna `battery_capacity`, como una lista donde cada elemento es una palabra de la cadena de texto original. Por ejemplo, $\texttt{"199 cc"}$ se convierte en la lista $\texttt{["199", "cc"]}$. Esto es lo que hace `split()`.

Al usar la instrucción `apply` sobre `separados` se espera que se aplique una función, en este caso, la función es $\texttt{x} \mapsto \texttt{x}[-1]$, es decir, recibe un elemento $\texttt{x}$, que en este caso es una lista, y devuelve $\texttt{x[-1]}$, su último elemento. Esto recupera el sufijo que nos interesa. El aplicar `unique()` nos muestra sólo los valores únicos resultantes, y podemos ver que existen varios sufijos distintos. Aquí, las opciones podrían ser:

* Dados todos los antecedentes que tenemos, podemos separar los autos eléctricos y no eléctricos, y purificar todas las observaciones que no terminen con $\texttt{kWh}$ y $\texttt{cc}$, teniendo en cuenta que existen variaciones de mayúsculas y minúsculas.
* Dejarlo como _string_ si los números de la columna no son tan relevantes para el problema. Incluso, se puede eliminar la columna si no aporta información significativa.

La que se decida depende del problema que se quiera resolver.



#### Caballos de potencia

Veamos qué pasa con la columna `hp`. Podemos reciclar el patrón de arriba, pero cambiando el final por $\texttt{hp}$.

In [ ]:
pat = r'^\s*(?:\d+|\d{1,3}(?:,\d{3})+)(?:\.\d+)?\s*hp\s*$'
match_condition = df["hp"].str.fullmatch(
    pat, case=False, na=False
  )

df[~match_condition].head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
254,Nissan,Urvan,2.5L Turbo Diese,NaN,2488 cc,140 km/h,14.0 sec,"$28,000",Diesel,15,356 Nm
270,Volkswagen,Golf,1.5L Turbo I4 / Electric,1498 cc / 40 - 58 kWh,110 - 320 hp,210 km/h,8.5 sec,"$23,000",Petrol,5,250 Nm
271,Volkswagen,Passat,2.0L Turbo I4,1984 cc,150 - 280 hp,210 km/h,7.8 sec,"$25,000",Petrol,5,250 Nm
272,Volkswagen,Tiguan,2.0L Turbo I4,1984 cc,150 - 230 hp,209 km/h,8.5 sec,"$28,505","Petrol, Hybrid",7,300 Nm


Tenemos otro problema: a veces se usa $\texttt{cc}$ en los caballos de potencia, y también existen rangos, como en las otras columnas (`torque`). Haremos un procedimiento similar al que hicimos con la capacidad de la batería para ver qué sufijos existen:

In [ ]:
df_sin_nulos = df[df['hp'].notna()]
separados = df_sin_nulos['hp'].str.split()
separados.apply(lambda x: x[-1]).unique()

array(['hp', 'cc', 'HP', '(est.)'], dtype=object)

Acá la variedad de sufijos es mucho menor. Existe tan sólo una variación que corresponde a $\texttt{cc}$. Como se verá en la sección donde se revisa la velocidad total, el sufijo $\texttt{(est.)}$ corresponde a una estimación, entonces no es necesariamente distinto. Una buena pregunta es: ¿cuántas columnas no calzan con el patrón que termina en $\texttt{hp}$?

In [ ]:
df[~match_condition].shape

(99, 11)

¡Son $99$! Dado que el _dataset_ tenía más de $1.000$ observaciones, podemos partir filtrando las que sí cumplen el patrón, es decir, tienen dígitos (usando o no la coma como separador de miles) seguidos del sufijo $\texttt{cc}$.

In [ ]:
def limpiar_hp(valor):
  # Borramos todas las comas del texto
  sin_comas = valor.replace(",", "")
  return sin_comas.strip().split(" ")[0]

df['hp'] = df['hp'].mask(match_condition, df['hp'].apply(limpiar_hp))
df.head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


Algo importante a mencionar es que la instrucción `mask` sirve para operar sólo donde cierta condición se cumple. En este caso, operamos donde `match_condition` sea $\texttt{True}$, es decir, donde el patrón calce. Al usar la librería `pandas`, el hacer referencia a una máscara (_mask_) es muy común. Una máscara es una condición de filtrado que aplicamos en los datos.

> ⚠️ **Advertencia**: La instrucción `mask` evalúa todas las filas antes de saber dónde aplicar la condición. Esto puede ser ineficiente, y en casos, incluso inducir a errores porque se requería un calce con un patrón muy estricto. Para evitar este comportamiento, les recomiendo usar la versión de más abajo con `df.loc`.

Ahora, tenemos que pensar qué hacemos con los rangos. Una decisión intuitiva podría ser calcular el promedio entre el límite inferior y el superior. Eso es lo que haremos en este tutorial para ejemplificar, pero la decisión debe estar fundamentada con algún estudio o incluso apoyándose en la definición de los rangos para esta columna, si es que existe en la documentación del _dataset_.

In [ ]:
df[df['hp'].str.contains('-')].shape

(81, 11)

Hay $81$ observaciones que preliminarmente tienen un rango (porque contienen un guion corto).

In [ ]:
pat = r'\d+-\d+'
guion_pegado = df["hp"].str.contains(pat, na=False)
guion_general = df["hp"].str.contains("-", na=False)

df[guion_pegado].head(), df[guion_general].head()

(     company_name        car_name         engines     battery_capacity  \
 2            Ford             KA+     1.2L Petrol             1,200 cc   
 1030         Ford  Mustang Mach-E  Electric Motor  Battery (68-98 kWh)   
 
               hp total_speed         performance             prices  \
 2       70-85 hp    165 km/h            10.5 sec    $12,000-$15,000   
 1030  290-480 hp    180 km/h  3.5 sec (GT Model)  $43,000 - $61,000   
 
      fuel_types seats             torque  
 2        Petrol     5       100 - 140 Nm  
 1030   Electric     5  430 Nm (GT Model)  ,
     company_name car_name                   engines       battery_capacity  \
 2           Ford      KA+               1.2L Petrol               1,200 cc   
 270   Volkswagen     Golf  1.5L Turbo I4 / Electric  1498 cc / 40 - 58 kWh   
 271   Volkswagen   Passat             2.0L Turbo I4                1984 cc   
 272   Volkswagen   Tiguan             2.0L Turbo I4                1984 cc   
 273   Volkswagen    Jetta 

Como se puede ver mirando la columna `hp`, hay que tener mucho cuidado. A veces los datos vienen con un guion corto que no está pegado a los números, por ejemplo, $\texttt{100 - 140} \space \text{hp}$, y a veces este guion sí está pegado. Hay que tratarlos como casos separados.

Primero, trataremos el caso donde el guion está pegado a los números.

In [ ]:
def limpiar_hp_rango_pegado(valor):
  sin_comas = valor.replace(",", "")
  rango = sin_comas.strip().split()[0]
  lim_inf, lim_sup = rango.split("-")
  return (int(lim_inf) + int(lim_sup)) / 2

df.loc[guion_pegado, 'hp'] = (
  df.loc[guion_pegado, 'hp'].apply(limpiar_hp_rango_pegado)
)

df.head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",77.5,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


Acá usamos la versión de reemplazar datos sólo evaluando en las observaciones que cumplan una condición con `df.loc` como se mencionó más arriba. El problema de usar la instrucción `mask` es que evalúa la función incluso en las filas que no cumplen la condición antes de aplicarla. Como en la función `limpiar_hp_rango_pegado` desempaquetamos los elementos de la lista `rango.split("-")` cuando definimos `lim_inf, lim_sup` en la misma línea, requerimos que exista un guion corto.

Un ejemplo para entender el desempaquetado es una asociación con dos elementos. Si tienen una lista $\texttt{L} = [1, 4]$ y quieren asignarle los elementos en ese orden a dos variables $\texttt{x}$ e $\texttt{y}$, simplemente desempaquetan dicha lista con $\texttt{x, y} = \texttt{L}$, entonces $\texttt{x}$ toma el valor $1$, e $\texttt{y}$ toma el valor $4$.

A continuación, haremos el procesamiento restante, pero para el caso donde el guion corto no está pegado a los números. Noten que si separamos la cadena de texto por espacios, bastaría acceder a las posiciones $0$ y $2$ de la lista rersultante para recuperar el límite inferior y superior, pues la posición $1$ tendrá el guion corto.

In [ ]:
mascara = guion_general & ~guion_pegado

def limpiar_hp_guion_separado(valor):
  sin_comas = valor.replace(",", "")
  separados_por_espacio = sin_comas.strip().split()
  lim_inf = separados_por_espacio[0]
  lim_sup = separados_por_espacio[2]
  return (int(lim_inf) + int(lim_sup)) / 2

df.loc[mascara, 'hp'] = (
  df.loc[mascara, 'hp'].apply(limpiar_hp_guion_separado)
)

df.head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",77.5,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


Acá, lo importante es que la máscara considera las observaciones que cumplen tener un guion corto (`guion_general`) y al mismo tiempo, ese guion corto no está pegado a los números (negación de `guion_pegado`).

Con la información que hemos recopilado hasta ahora, podemos saber cuántas observaciones nos faltan por tratar. Sin embargo, esta información la recuperaremos viendo cuántas celdas aún no son números.

In [ ]:
def es_numero(texto):
  if isinstance(texto, int) or isinstance(texto, float):
    return True
  texto = texto.strip()
  return texto.isdigit()

df[~df['hp'].apply(es_numero)].head(10)

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
254,Nissan,Urvan,2.5L Turbo Diese,NaN,2488 cc,140 km/h,14.0 sec,"$28,000",Diesel,15,356 Nm
328,Volkswagen,ID.2,Electric,~50 kWh (Estimate),~170 hp,160 km/h,8.0 sec,"$25,000",Electric,5,220 Nm
629,Tata Motors,Altroz,1.2L Revotron Petrol / 1.5L Diesel,1199 cc / 1497 cc,86 hp / 89 hp,170 km/h,12 sec,"$10,400",Petrol/Diesel,5,113-200 Nm
639,Tata Motors,Zest,1.3L Quadrajet Diesel / 1.2L Revotron Petrol,1248 cc / 1199 cc,75 hp / 90 hp,165 km/h,15 sec,"$8,700",Diesel/Petrol,5,190 Nm / 140 Nm
642,Tata Motors,Indigo Marina,1.4L CR4 Diesel / 1.4L Petrol,1396 cc,70 hp / 85 hp,150 km/h,17 sec,"$8,300",Diesel/Petrol,5,135 Nm / 115 Nm
643,Tata Motors,Manza,1.3L Quadrajet Diesel / 1.4L Petrol,1248 cc / 1368 cc,75 hp / 90 hp,160 km/h,14 sec,"$9,000",Diesel/Petrol,5,200 Nm / 115 Nm
647,Tata Motors,Nexon,1.2L Turbo Petrol / 1.5L Diesel,1199 cc / 1497 cc,118 hp / 110 hp,180 km/h,10 sec,"$14,000",Petrol/Diesel,5,170-260 Nm
794,GMC,Hummer EV SUV Edition 1,"Electric motors with up to 11,500 lb-ft torque",200 kWh,Up to 830 hp,170 km/h,3.5 sec,"$96,550",Electric,5,"15,590 Nm"
795,GMC,Hummer EV Pickup Edition 1,"Electric motors with up to 11,500 lb-ft torque",200 kWh,"Up to 1,000 hp",170 km/h,3.0 sec,"$96,550",Electric,5,"15,590 Nm"
809,GMC,Hummer EV SUV Extreme Off-Road,"Electric motors with up to 11,500 lb-ft torque",200 kWh,Up to 830 hp,170 km/h,3.6 sec,"$110,000",Electric,5,"15,590 Nm"


Para la función `es_numero`, podemos mencionar las siguientes instrucciones nuevas:

- `isinstance` permite saber si una variable cumple ser de una clase o no. En este caso, si cumple ser de la clase `int` (número entero) o `float` (número decimal), entonces sí es un número, por lo tanto, se devuelve $\texttt{True}$.
- `isdigit()` permite saber si todos los caracteres de una palabra son dígitos.

El caso de revisar un decimal como texto no es necesario, porque al aplicar los dos casos de rangos ya dejamos el resultado como un número, pues se devolvía una división aritmética.

Como se puede ver, los casos restantes son más de lo mismo, entonces pueden seguir aplicando las técnicas que hemos visto en este tutorial según las necesidades de su problema.

#### Velocidad total

Realizaremos un procedimiento similar con `total_speed`, pero reemplazando el final del patrón por $\texttt{km/h}$:

In [ ]:
pat = r'^\s*(?:\d+|\d{1,3}(?:,\d{3})+)(?:\.\d+)?\s*km/h\s*$'
match_condition = df["total_speed"].str.fullmatch(
    pat, case=False, na=False
  )

df[~match_condition].head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
1146,Mazda,RX-Vision Concept,Rotary SkyActiv-R,NaN,400 hp (est.),300 km/h (est.),NaN,N/A (Concept Only),Petrol (Hybrid),2,NaN


Esto es mucho mejor: sólo una fila no cumple la condición, y además, lo podemos arreglar recuperando el número antes del sufijo. Así, `total_speed` es un buen ejemplo de una columna numérica. Recuperemos el prefijo. Para ello, definiremos una función que será aplicada a toda la columna:

In [ ]:
def limpiar_kmh(valor):
  sin_comas = valor.replace(",", "")
  return sin_comas.strip().split(" ")[0]

df['total_speed'] = df['total_speed'].apply(limpiar_kmh)
df.head()

,company_name,car_name,engines,battery_capacity,hp,total_speed,performance,prices,fuel_types,seats,torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963,340,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563,250,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",77.5,165,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630,250,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602,320,3.6 sec,"$253,290",Petrol,2,560 Nm


La función aprovecha que el formato de la columna es $\texttt{<digitos> <sufijo...>}$ para separar por espacios (`split(" ")`) y extraer el primer elemento. El método `split(...)` permite transformar una cadena de texto a una lista separando por el caracter que se pase como argumento. Ahora, ya podemos transformar esta columna a tipo `int` (número entero).

In [ ]:
df['total_speed'] = df['total_speed'].astype(int)
df.dtypes

,0
company_name,string[python]
car_name,string[python]
engines,string[python]
battery_capacity,object
hp,object
total_speed,int64
performance,object
prices,object
fuel_types,string[python]
seats,object


Esto se puede seguir haciendo con las columnas que se analizaron en este tutorial. Para complementar la parte de estadística y visualización, recomiendo mucho [este tutorial](https://www.kaggle.com/code/abonaplata/analisis-exploratorio-de-datos-con-python) y lo que vimos en la [clase 4](https://www.u-cursos.cl/escverano/2025/1/FM849/1/material_docente/detalle?id=9550237).

Ante cualquier duda, me pueden escribir por el correo de U-Cursos, o el que se indica al inicio de este tutorial. ¡Mucho éxito!